# NeurIPS : notebook by pragnyanramtha

In [8]:
# ===================================================================
# 1.1: Project Setup and Dependencies
# File: setup.py
# ===================================================================
import os
import logging
import torch
import warnings

def setup_project_environment():
    """
    Creates directories, installs packages, and configures logging.
    """
    # --- Directory Creation ---
    print("Creating project directories...")
    directories = ['data', 'notebooks', 'src/data_processing', 'src/models', 
                   'src/evaluation', 'src/utils', 'results/models', 'results/submissions']
    for directory in directories:
        os.makedirs(directory, exist_ok=True)
    
    # --- Package Installation ---
    # In a real environment, you would run this in your terminal.
    # We will list the command here for completeness.
    print("\n---")
    print("Run the following command in your terminal to install dependencies:")
    pip_install_command = ("pip install pandas numpy scikit-learn xgboost catboost lightgbm "
                           "tabpfn torch torchvision torchaudio matplotlib seaborn pyarrow fastparquet")
    print(f"$ {pip_install_command}")
    print("---\n")

    # --- CUDA Configuration ---
    print("Checking for CUDA support...")
    is_cuda_available = torch.cuda.is_available()
    print(f"CUDA Available: {is_cuda_available}")
    if not is_cuda_available:
        print("WARNING: CUDA not found. Training will be on CPU.")
    
    # --- Logging and Warnings Configuration ---
    print("Configuring logging...")
    logging.basicConfig(level=logging.INFO, 
                        format='%(asctime)s - %(levelname)s - %(message)s',
                        filename='project_log.log',
                        filemode='w')
    
    # Suppress common warnings for cleaner output
    warnings.filterwarnings('ignore', category=FutureWarning)
    
    print("\nProject setup complete.")
    logging.info("Project environment set up successfully.")

# Execute the setup
setup_project_environment()

Creating project directories...

---
Run the following command in your terminal to install dependencies:
$ pip install pandas numpy scikit-learn xgboost catboost lightgbm tabpfn torch torchvision torchaudio matplotlib seaborn pyarrow fastparquet
---

Checking for CUDA support...
CUDA Available: True
Configuring logging...

Project setup complete.


## Core Data Loading and Preprocessing

This section contains the functions responsible for interacting with the raw data. We begin by creating a robust data loader that handles the large parquet files, performs the critical ADC conversion to restore the data's dynamic range, and includes error handling. We then implement functions to apply the various calibration frames, such as dark subtraction and flat-field correction, to clean the instrumental signatures from the signal.

In [9]:
# ===================================================================
# 2.1 & 2.2: Data Loading and Calibration
# File: src/data_processing/loader.py
# ===================================================================

import numpy as np
from pathlib import Path
import cupy as cp  # Import CuPy
import pandas as pd
from pathlib import Path
import logging


# ===================================================================
# 2.1 & 2.2: Data Loading and Calibration
# File: src/data_processing/loader.py
# ===================================================================
import pandas as pd
import numpy as np
from pathlib import Path

# --- 2.1: Data Loading Utilities ---

def load_adc_info(data_path):
    """Loads ADC conversion parameters."""
    return pd.read_csv(Path(data_path) / 'adc_info.csv').iloc[0]

def load_signal_data(file_path, adc_params, instrument):
    """Loads a single signal parquet file and applies ADC conversion."""
    try:
        df = pd.read_parquet(file_path)
        raw_signal = df.to_numpy()
        
        # Determine shape based on instrument
        if instrument == 'FGS1':
            reshaped_signal = raw_signal.reshape(-1, 32, 32)
        elif instrument == 'AIRS-CH0':
            reshaped_signal = raw_signal.reshape(-1, 32, 356)
        else:
            raise ValueError("Unknown instrument")
            
        # Apply ADC conversion
        gain = adc_params['gain']
        offset = adc_params['offset']
        signal_float64 = (reshaped_signal / gain + offset).astype(np.float64)
        
        logging.info(f"Successfully loaded and converted {file_path}")
        return signal_float64
    
    except Exception as e:
        logging.error(f"Failed to load or process {file_path}: {e}")
        return None

# --- 2.2: Calibration Data Processing ---

def load_calibration_files(planet_path, instrument, visit):
    """Loads all calibration files for a given instrument and visit."""
    calib_path = Path(planet_path) / f"{instrument}_calibration_{visit}"
    calib_data = {}
    for calib_type in ['dark', 'flat', 'dead', 'linear_corr', 'read']:
        file_path = calib_path / f"{calib_type}.parquet"
        if file_path.exists():
            calib_data[calib_type] = pd.read_parquet(file_path).to_numpy()
    return calib_data

def apply_calibrations(signal_data, calib_data):
    """Applies a simplified calibration pipeline."""
    # This is a simplified example. A real pipeline would be more complex.
    processed_signal = signal_data
    if 'dark' in calib_data:
        processed_signal = processed_signal - calib_data['dark']
    if 'flat' in calib_data:
        # Avoid division by zero
        flat = calib_data['flat']
        flat[flat == 0] = 1
        processed_signal = processed_signal / flat
    
    # Dead pixel masking could be applied here by setting values to NaN or interpolating
    return processed_signal

# --- 2.3: Multi-Visit Combination ---

def process_all_planet_visits(planet_path, adc_params):
    """
    Loads all data for a single planet, handles multiple visits, 
    and applies calibrations.
    """
    planet_path = Path(planet_path)
    processed_data = {'FGS1': [], 'AIRS-CH0': []}

    for instrument in ['FGS1', 'AIRS-CH0']:
        visit_files = sorted(list(planet_path.glob(f'{instrument}_signal_*.parquet')))
        
        for visit_file in visit_files:
            visit_id = visit_file.stem.split('_')[-1]
            
            # Load signal data
            signal_data = load_signal_data(visit_file, adc_params, instrument)
            if signal_data is None: continue
            
            # Load corresponding calibration data
            calib_data = load_calibration_files(planet_path, instrument, visit_id)
            
            # Apply calibrations
            calibrated_signal = apply_calibrations(signal_data, calib_data)
            processed_data[instrument].append(calibrated_signal)
            
    # Combine visits by concatenating along the time axis
    for instrument in processed_data:
        if processed_data[instrument]:
            processed_data[instrument] = np.concatenate(processed_data[instrument], axis=0)
        else:
            processed_data[instrument] = np.array([])
            
    return processed_data
# We can reuse the original ADC loader
# from src.data_processing.loader import load_adc_info


def load_adc_info(data_path):
    """Loads ADC conversion parameters."""
    return pd.read_csv(Path(data_path) / 'adc_info.csv').iloc[0]

def load_and_move_to_gpu(file_path, instrument):
    """Loads a parquet file and immediately moves its data to the GPU."""
    try:
        df = pd.read_parquet(file_path)
        # Move data to GPU as a CuPy array
        raw_signal_gpu = cp.asarray(df.to_numpy()) 
        
        if instrument == 'FGS1':
            return raw_signal_gpu.reshape(-1, 32, 32)
        elif instrument == 'AIRS-CH0':
            return raw_signal_gpu.reshape(-1, 32, 356)
        else:
            raise ValueError("Unknown instrument")
            
    except Exception as e:
        logging.error(f"GPU Load Failed for {file_path}: {e}")
        return None


def apply_adc_conversion_gpu(signal_gpu, adc_params, instrument):
    """Applies ADC conversion on the GPU using instrument-specific keys."""
    # --- FIX ---
    # Dynamically create the key for the specific instrument
    gain_key = f"{instrument}_adc_gain"
    offset_key = f"{instrument}_adc_offset"

    # Look up the correct gain and offset
    gain = adc_params[gain_key]
    offset = adc_params[offset_key]
    
    # The rest of the logic is the same
    return (signal_gpu / gain + offset).astype(cp.float64)


def apply_calibrations_gpu(signal_gpu, calib_data_gpu):
    """Applies a simplified calibration pipeline on the GPU."""
    processed_signal_gpu = signal_gpu
    if 'dark' in calib_data_gpu:
        processed_signal_gpu = processed_signal_gpu - calib_data_gpu['dark']
    if 'flat' in calib_data_gpu:
        flat_gpu = calib_data_gpu['flat']
        # Avoid division by zero on the GPU
        flat_gpu[flat_gpu == 0] = 1
        processed_signal_gpu = processed_signal_gpu / flat_gpu
    return processed_signal_gpu

def create_light_curve_gpu(signal_data_gpu):
    """Creates a 1D light curve on the GPU."""
    if signal_data_gpu.ndim != 3:
        return cp.array([])
    # cp.sum is the CuPy equivalent of np.sum
    return cp.sum(signal_data_gpu, axis=(1, 2))

def extract_temporal_features_gpu(light_curve_gpu):
    """Extracts basic statistical features on the GPU."""
    if light_curve_gpu.size == 0:
        # Return values must be moved to CPU for the final dictionary
        return {'mean': 0.0, 'std': 0.0, 'min': 0.0, 'max': 0.0}
    
    # cp.asnumpy() moves the final scalar result from GPU to CPU
    return {
        'mean': cp.asnumpy(cp.mean(light_curve_gpu)),
        'std': cp.asnumpy(cp.std(light_curve_gpu)),
        'min': cp.asnumpy(cp.min(light_curve_gpu)),
        'max': cp.asnumpy(cp.max(light_curve_gpu))
    }

def process_planet_gpu(planet_dir, adc_params, star_info_df):
    """
    Main GPU processing function for a single planet.
    Orchestrates loading, calibration, and feature extraction on the GPU.
    """
    planet_path = Path(planet_dir)
    planet_id = int(planet_path.name)
    features = {'planet_id': planet_id}

    for instrument in ['FGS1', 'AIRS-CH0']:
        visit_files = sorted(list(planet_path.glob(f'{instrument}_signal_*.parquet')))
        if not visit_files:
            # Handle cases where an instrument file might be missing
            temp_features = extract_temporal_features_gpu(cp.array([]))
            for key, val in temp_features.items():
                features[f'{instrument}_{key}'] = val
            continue

        all_visits_gpu = []
        for visit_file in visit_files:
            visit_id = visit_file.stem.split('_')[-1]
            
            # Load raw signal directly to GPU
            signal_gpu = load_and_move_to_gpu(visit_file, instrument)
            if signal_gpu is None: continue

            # Apply ADC on GPU
            signal_gpu = apply_adc_conversion_gpu(signal_gpu, adc_params,instrument)
            
            # Load and apply calibrations on GPU
            calib_path = planet_path / f"{instrument}_calibration_{visit_id}"
            calib_data_gpu = {}
            for calib_type in ['dark', 'flat']: # Simplified for example
                calib_file = calib_path / f"{calib_type}.parquet"
                if calib_file.exists():
                    # Load and move calibration frame to GPU
                    calib_data_gpu[calib_type] = cp.asarray(pd.read_parquet(calib_file).to_numpy())
            
            calibrated_signal_gpu = apply_calibrations_gpu(signal_gpu, calib_data_gpu)
            all_visits_gpu.append(calibrated_signal_gpu)

        # Combine visits and extract features
        if all_visits_gpu:
            full_signal_gpu = cp.concatenate(all_visits_gpu, axis=0)
            light_curve_gpu = create_light_curve_gpu(full_signal_gpu)
            temp_features = extract_temporal_features_gpu(light_curve_gpu)
        else:
            temp_features = extract_temporal_features_gpu(cp.array([]))
        
        for key, val in temp_features.items():
            features[f'{instrument}_{key}'] = val

    # Add stellar features (this is a CPU operation)
    stellar_params = star_info_df[star_info_df['planet_id'] == planet_id].iloc[0]
    features.update(stellar_params.to_dict())
    
    return features

# --- 2.1: Data Loading Utilities ---

# ===================================================================
# File: src/models/boosting_gpu.py
# ===================================================================
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from sklearn.model_selection import train_test_split
import sklearn.multioutput
import numpy as np

# Assuming the evaluation metric function `calculate_weighted_gll` is in another file
# from src.evaluation.metrics import calculate_weighted_gll

def train_and_evaluate_boosting_gpu(X, y):
    """
    Trains and evaluates GPU-accelerated XGBoost, LightGBM, and CatBoost models.
    """
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
    print("\n--- Training GPU-Accelerated Boosting Models ---")
    
    # --- 1. XGBoost ---
    print("\nTraining XGBoost...")
    # Use MultiOutputRegressor wrapper for multi-target prediction
    xgb_model = sklearn.multioutput.MultiOutputRegressor(
        xgb.XGBRegressor(tree_method='gpu_hist', objective='reg:squarederror', n_estimators=1000)
    )
    xgb_model.fit(X_train, y_train) # XGBoost can handle numpy arrays directly
    
    y_pred_mean_xgb = xgb_model.predict(X_val)
    # Simple uncertainty: std of training residuals
    unc_xgb = np.std(y_train - xgb_model.predict(X_train), axis=0)
    score_xgb = calculate_weighted_gll(y_val, y_pred_mean_xgb, unc_xgb)
    print(f"XGBoost Validation GLL: {score_xgb:.4f}")

    # --- 2. LightGBM ---
    print("\nTraining LightGBM...")
    lgb_model = sklearn.multioutput.MultiOutputRegressor(
        lgb.LGBMRegressor(device='gpu', objective='regression', n_estimators=1000)
    )
    lgb_model.fit(X_train, y_train)
    
    y_pred_mean_lgb = lgb_model.predict(X_val)
    unc_lgb = np.std(y_train - lgb_model.predict(X_train), axis=0)
    score_lgb = calculate_weighted_gll(y_val,data_pred_mean_lgb, unc_lgb)
    print(f"LightGBM Validation GLL: {score_lgb:.4f}")

    # --- 3. CatBoost ---
    print("\nTraining CatBoost...")
    # CatBoost's Multi-output support is native but can be tricky. Using the wrapper is safer.
    cat_model = sklearn.multioutput.MultiOutputRegressor(
        cb.CatBoostRegressor(task_type='GPU', iterations=1000, verbose=0)
    )
    cat_model.fit(X_train, y_train)
    
    y_pred_mean_cat = cat_model.predict(X_val)
    unc_cat = np.std(y_train - cat_model.predict(X_train), axis=0)
    score_cat = calculate_weighted_gll(y_val, y_pred_mean_cat, unc_cat)
    print(f"CatBoost Validation GLL: {score_cat:.4f}")
    
    return {'xgb': xgb_model, 'lgb': lgb_model, 'cat': cat_model}



# ===================================================================
# File: src/models/pytorch_nn.py
# ===================================================================
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# --- 2.1: PyTorch Dataset and DataLoader ---

def create_dataloaders(X, y, batch_size=64):
    """Creates PyTorch DataLoaders for training and validation sets."""
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Convert numpy arrays to PyTorch tensors
    X_train_t = torch.tensor(X_train, dtype=torch.float32)
    y_train_t = torch.tensor(y_train, dtype=torch.float32)
    X_val_t = torch.tensor(X_val, dtype=torch.float32)
    y_val_t = torch.tensor(y_val, dtype=torch.float32)
    
    train_dataset = TensorDataset(X_train_t, y_train_t)
    val_dataset = TensorDataset(X_val_t, y_val_t)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader

# --- 2.2: PyTorch Model Architecture ---

class SimpleMLP(nn.Module):
    """A simple Multi-Layer Perceptron for regression."""
    def __init__(self, input_dim, output_dim):
        super(SimpleMLP, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, output_dim)
        )
        
    def forward(self, x):
        return self.network(x)

# --- 2.3: PyTorch Training and Evaluation Loop ---

def train_and_evaluate_pytorch_model(X, y, epochs=20):
    """
    Main function to orchestrate the training and evaluation of a PyTorch model.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"\n--- Training PyTorch Model on {device} ---")
    
    # Create DataLoaders
    train_loader, val_loader = create_dataloaders(X, y)
    
    # Initialize model, loss, and optimizer
    input_dim = X.shape[1]
    output_dim = y.shape[1]
    model = SimpleMLP(input_dim, output_dim).to(device)
    criterion = nn.MSELoss() # A common loss for regression
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    # Training loop
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {total_loss / len(train_loader):.6f}")

    # Evaluation
    model.eval()
    all_preds = []
    all_true = []
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X = batch_X.to(device)
            outputs = model(batch_X)
            all_preds.append(outputs.cpu().numpy())
            all_true.append(batch_y.numpy())
            
    y_pred_mean_nn = np.concatenate(all_preds)
    y_val = np.concatenate(all_true)
    
    # Simple uncertainty estimation
    unc_nn = np.std(y - model(torch.tensor(X, dtype=torch.float32).to(device)).detach().cpu().numpy(), axis=0)
    score_nn = calculate_weighted_gll(y_val, y_pred_mean_nn, unc_nn)
    print(f"PyTorch MLP Validation GLL: {score_nn:.4f}")
    
    return model
# --- 2.3: Multi-Visit Combination ---

def process_all_planet_visits(planet_path, adc_params):
    """
    Loads all data for a single planet, handles multiple visits, 
    and applies calibrations.
    """
    planet_path = Path(planet_path)
    processed_data = {'FGS1': [], 'AIRS-CH0': []}

    for instrument in ['FGS1', 'AIRS-CH0']:
        visit_files = sorted(list(planet_path.glob(f'{instrument}_signal_*.parquet')))
        
        for visit_file in visit_files:
            visit_id = visit_file.stem.split('_')[-1]
            
            # Load signal data
            signal_data = load_signal_data(visit_file, adc_params, instrument)
            if signal_data is None: continue
            
            # Load corresponding calibration data
            calib_data = load_calibration_files(planet_path, instrument, visit_id)
            
            # Apply calibrations
            calibrated_signal = apply_calibrations(signal_data, calib_data)
            processed_data[instrument].append(calibrated_signal)
            
    # Combine visits by concatenating along the time axis
    for instrument in processed_data:
        if processed_data[instrument]:
            processed_data[instrument] = np.concatenate(processed_data[instrument], axis=0)
        else:
            processed_data[instrument] = np.array([])
            
    return processed_data

## Comprehensive Feature Engineering Pipeline

With the data loaded and cleaned, we now focus on feature engineering. The strategy is to reduce the dimensionality of the vast time-series data into a compact and informative feature vector. We perform simple aperture photometry to create 1D light curves, extract basic statistical features from these light curves, and combine them with the scaled stellar parameters to form the final input for our models.

In [10]:
# ===================================================================
# 3.1, 3.2 & 3.3: Feature Engineering
# File: src/feature_engineering/builder.py
# ===================================================================
from sklearn.preprocessing import StandardScaler

def create_light_curve(signal_data):
    """
    Creates a simplified 1D light curve from 3D signal data by summing
    all pixel values for each timestamp. This is a basic form of photometry.
    """
    if signal_data.ndim != 3:
        return np.array([])
    # Sum across the spatial dimensions (height and width)
    return np.sum(signal_data, axis=(1, 2))

def extract_temporal_features(light_curve):
    """Extracts basic statistical features from a light curve."""
    if light_curve.size == 0:
        return {'mean': 0, 'std': 0, 'min': 0, 'max': 0}
    
    return {
        'mean': np.mean(light_curve),
        'std': np.std(light_curve),
        'min': np.min(light_curve),
        'max': np.max(light_curve)
    }

def get_stellar_features(planet_id, star_info_df):
    """Retrieves stellar parameters for a given planet_id."""
    return star_info_df[star_info_df['planet_id'] == planet_id].iloc[0]

def build_feature_vector(planet_id, all_calibrated_data, star_info_df):
    """
    Builds a single feature vector for a planet by combining temporal
    and stellar features.
    """
    features = {'planet_id': planet_id}
    
    # Temporal features from light curves
    for instrument in ['FGS1', 'AIRS-CH0']:
        light_curve = create_light_curve(all_calibrated_data[instrument])
        temp_features = extract_temporal_features(light_curve)
        for key, val in temp_features.items():
            features[f'{instrument}_{key}'] = val
            
    # Stellar features
    stellar_params = get_stellar_features(planet_id, star_info_df)
    features.update(stellar_params.to_dict())
    
    return features

## Baseline Models and Evaluation

This section establishes our modeling and evaluation framework. We implement the competition-specific Gaussian Log-Likelihood (GLL) metric, ensuring the heavy FGS1 channel weight is correctly applied. We then create a function to train a simple baseline model, such as Ridge regression, and a validation function to score its performance using a standard train-test split.

In [11]:
# ===================================================================
# 4.1 & 4.2: Baseline Models and Evaluation
# File: src/models/baseline.py and src/evaluation/metrics.py
# ===================================================================
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import make_scorer
import sklearn.multioutput

# --- 4.2: Evaluation Framework ---

def gll_score_single(y_true, y_pred_mean, y_pred_unc):
    """Calculates the Gaussian Log-Likelihood for a single prediction."""
    return -0.5 * (np.log(2 * np.pi) + np.log(y_pred_unc**2) + ((y_true - y_pred_mean)**2) / (y_pred_unc**2))

def calculate_weighted_gll(y_true, y_pred_mean, y_pred_unc, fgs1_weight=57.846):
    """Calculates the final weighted GLL score for the competition."""
    # Ensure inputs are numpy arrays
    y_true = np.asarray(y_true)
    y_pred_mean = np.asarray(y_pred_mean)
    y_pred_unc = np.asarray(y_pred_unc)
    
    scores = gll_score_single(y_true, y_pred_mean, y_pred_unc)
    
    # Apply weights
    weights = np.ones(y_true.shape[1])
    weights[0] = fgs1_weight  # First column is FGS1
    
    weighted_scores = scores * weights
    return np.sum(weighted_scores)


# --- 4.1: Simple Regression Baselines ---

def train_and_evaluate_baseline(X, y):
    """Trains a Ridge Regressor and evaluates it using the GLL score."""
    
    # Split data
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
    print(f"Training on {X_train.shape[0]} samples, validating on {X_val.shape[0]} samples.")
    
    # Initialize and train a simple model
    # We use a MultiOutputRegressor to predict all 283 wavelengths at once
    model = sklearn.multioutput.MultiOutputRegressor(Ridge(alpha=1.0))
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred_mean = model.predict(X_val)
    
    # Estimate uncertainty: A simple approach is to use the standard deviation
    # of the training residuals as a constant uncertainty for all predictions.
    train_residuals = y_train - model.predict(X_train)
    y_pred_unc = np.std(train_residuals, axis=0)
    
    # Evaluate
    score = calculate_weighted_gll(y_val, y_pred_mean, y_pred_unc)
    print(f"Validation Weighted GLL Score: {score:.4f}")
    
    return model, score

## Executing the Pipeline

Finally, this is the main execution script. It orchestrates the entire process by calling the functions defined above in the correct sequence. It iterates through all training planets, applies the data processing and feature engineering steps, aggregates the results into a single dataset, and then trains and evaluates our baseline model.

In [ ]:
# ===================================================================
# File: main.py (Updated Version)
# ===================================================================
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import logging


def main_gpu():
    """Main function to run the GPU-accelerated pipeline."""
    BASE_PATH = Path('/kaggle/input/ariel-data-challenge-2025')
    TRAIN_PATH = BASE_PATH / 'train'
    
    logging.info("Starting GPU-accelerated pipeline execution.")
    
    # --- Load Metadata (on CPU) ---
    print("Loading metadata...")
    adc_params = load_adc_info(BASE_PATH)
    star_info_df = pd.read_csv(BASE_PATH / 'train_star_info.csv')
    ground_truth_df = pd.read_csv(BASE_PATH / 'train.csv')
    
    # --- GPU-Accelerated Feature Engineering Loop ---
    print("Building feature vectors for all training planets using CuPy...")
    planet_dirs = [d for d in TRAIN_PATH.iterdir() if d.is_dir()]
    
    
    all_features = []
    # The main loop now calls the GPU processing function
    for planet_dir in tqdm(planet_dirs, desc="GPU Processing Planets"):
        feature_vector = process_planet_gpu(planet_dir, adc_params, star_info_df)
        all_features.append(feature_vector)
    
    feature_df = pd.DataFrame(all_features)
    print("GPU feature engineering complete.")
    
    # --- Model Training (unchanged from before) ---
    print("Preparing data for model training...")
    merged_df = pd.merge(feature_df, ground_truth_df, on='planet_id', how='inner')
    
    feature_cols = [col for col in feature_df.columns if col != 'planet_id']
    target_cols = [col for col in ground_truth_df.columns if col.startswith('wl_')]
    
    X = merged_df[feature_cols].values
    y = merged_df[target_cols].values
    
    # It's good practice to scale features for NNs and some boosting models
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # --- Run GPU-Accelerated Models ---
    train_and_evaluate_boosting_gpu(X_scaled, y)
    train_and_evaluate_pytorch_model(X_scaled, y)
    
    logging.info("GPU pipeline execution finished.")

# --- Run the main GPU pipeline ---
if __name__ == '__main__':
    # Ensure you have CuPy installed and a compatible GPU
    try:
        import cupy
        print(f"CuPy installation found. Running on GPU.")
        main_gpu()
    except ImportError:
        print("CuPy not found. Please install CuPy to run the GPU-accelerated pipeline.")
        # Optionally, you could fall back to the CPU main function here.

CuPy installation found. Running on GPU.
Loading metadata...
Building feature vectors for all training planets using CuPy...


GPU Processing Planets:   5%|▍         | 52/1100 [03:48<47:11,  2.70s/it]  